In [1]:
%load_ext autoreload

In [40]:
import numpy as np
import os
import mcubes
import meshcat
import pydrake
from pydrake.geometry import SceneGraph
from pydrake.systems.framework import DiagramBuilder
from pydrake.common import FindResourceOrThrow
from pydrake.multibody.plant import MultibodyPlant, AddMultibodyPlantSceneGraph
from pydrake.multibody.parsing import Parser, LoadModelDirectives, ProcessModelDirectives
from pydrake.multibody.tree import RevoluteJoint
from pydrake.all import ConnectMeshcatVisualizer, InverseKinematics, RigidTransform
from pydrake.all import BsplineTrajectoryThroughUnionOfHPolyhedra, IrisInConfigurationSpace, IrisOptions
import time
from meshcat import Visualizer
from functools import partial

import ipywidgets as widgets
from IPython.display import display

from meshcat_cpp_utils import StartMeshcat, MeshcatJointSliders
from pydrake.all import MeshcatVisualizerCpp, MeshcatVisualizerParams, Role

In [41]:
# Setup meshcat
from meshcat.servers.zmqserver import start_zmq_server_as_subprocess
proc, zmq_url, web_url = start_zmq_server_as_subprocess(server_args=[])
proc2, zmq_url2, web_url2 = start_zmq_server_as_subprocess(server_args=[])

In [64]:
#settings
q0 = [0.0, 0.0, 0.0]
q_low = [-1.7,-1.7, -2.0]
q_high = [1.7, 1.7, 2.0]

#marching cubes
q_low_mc = q_low.copy()
q_high_mc =  q_high.copy()
N = 20

In [65]:
vis = Visualizer(zmq_url=zmq_url)
vis.delete()
vis2 = Visualizer(zmq_url=zmq_url2)
vis2.delete()
#display(vis.jupyter_cell())

builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.001)
parser = Parser(plant)
tworob_asset = FindResourceOrThrow("drake/sandbox/assets/doublerob.urdf")
#two_dof_asset = FindResourceOrThrow("drake/sandbox/assets/planar2dof.urdf")
box_asset = FindResourceOrThrow("drake/sandbox/assets/box.urdf")

models =[]
models.append(parser.AddModelFromFile(tworob_asset))
#models.append(parser.AddModelFromFile(one_dof_asset))
models.append(parser.AddModelFromFile(box_asset))

idx = 0
for model in models:
    for joint_index in plant.GetJointIndices(model):
        joint = plant.get_mutable_joint(joint_index)
        if isinstance(joint, RevoluteJoint):
            joint.set_default_angle(q0[idx])
            idx += 1
            
locs = [[0.,0.,0.],[0.,0.,0.]]
idx = 0
for model in models:
    plant.WeldFrames(plant.world_frame(), plant.GetFrameByName("base", model), RigidTransform(locs[idx]))
    idx+=1

plant.Finalize()

visualizer = ConnectMeshcatVisualizer(builder, scene_graph, zmq_url=zmq_url, delete_prefix_on_load=False, )

diagram = builder.Build()
visualizer.load()
context = diagram.CreateDefaultContext()
plant_context = plant.GetMyContextFromRoot(context)
diagram.Publish(context)

joints = []
idx = 0
for model in models:
    jointindx = plant.GetJointIndices(model)
    for j in jointindx:
        joint = plant.get_mutable_joint(j)
        if isinstance(joint, RevoluteJoint):
            joints.append(joint)
            joints[-1].set_position_limits(lower_limits= np.array([q_low[idx]]), upper_limits= np.array([np.array([q_high[idx]])]))
            idx +=1
        
    
def set_joint_ang(val, idx):
    joints[idx].set_angle(plant_context, val)
    
ik = InverseKinematics(plant, plant_context)
collision_constraint = ik.AddMinimumDistanceConstraint(0.001, 0.01)

def eval_cons(q0, q1, q2, c, tol):
        return 1-1*float(c.evaluator().CheckSatisfied([q0, q1, q2], tol))
    
col_func_handle = partial(eval_cons, c=collision_constraint, tol=0.01)

def showres(q):
    set_joint_ang(q[0],0)
    set_joint_ang(q[1],1)
    set_joint_ang(q[2],2)
    vis2["q"].set_object(
            meshcat.geometry.Sphere(0.2), meshcat.geometry.MeshLambertMaterial(color=0xFFB900))
    vis2["q"].set_transform(
            meshcat.transformations.translation_matrix(q))
    diagram.Publish(context)
    print("              ", end = "\r")
    print(col_func_handle(*q) , end = "\r")

sliders = []
sliders.append(widgets.FloatSlider(min=q_low[0], max=q_high[0], value=0, description='q0'))
sliders.append(widgets.FloatSlider(min=q_low[1], max=q_high[1], value=0, description='q1'))
sliders.append(widgets.FloatSlider(min=q_low[2], max=q_high[2], value=0, description='q2'))

q = q0.copy()
def handle_slider_change(change, idx):
    q[idx] = change['new']
    #print(q, end="\r")
    showres(q)
    
idx = 0
for slider in sliders:
    slider.observe(partial(handle_slider_change, idx = idx), names='value')
    idx+=1


You can open the visualizer by visiting the following URL:
http://127.0.0.1:7009/static/
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7010/static/
Connecting to meshcat-server at zmq_url=tcp://127.0.0.1:6009...
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7009/static/
Connected to meshcat-server.


In [66]:
#marching cubes
vertices, triangles = mcubes.marching_cubes_func(tuple(q_low), tuple(q_high), N, N, N, col_func_handle, 0.5)
vis2["collision_constraint"].set_object(
            meshcat.geometry.TriangularMeshGeometry(vertices, triangles),
            meshcat.geometry.MeshLambertMaterial(color=0xff0000, wireframe=False))
q = q0.copy()
showres(q)

In [67]:
for slider in sliders:
    display(slider)

display(vis.jupyter_cell())
display(vis2.jupyter_cell())

Widget Javascript not detected.  It may not be installed or enabled properly.


Widget Javascript not detected.  It may not be installed or enabled properly.


Widget Javascript not detected.  It may not be installed or enabled properly.


In [ ]:
plant.SetPo

In [68]:
seed_points = np.array([[0.0, 0, 0], # startpoint
                        [0.8, -0.8, 1.3],  # blue low green up
                        [0.1, -1.7, 0.9],     # green low other up
                        [-0.5, -1.0, 1.9]])    # passing


#run iris here :)))
iris_options = IrisOptions()
iris_options.require_sample_point_is_contained = True
iris_options.iteration_limit = 10
iris_options.enable_ibex = False

regions = []
for i in range(seed_points.shape[0]):
    start_time = time.time()
    hpoly = IrisInConfigurationSpace(plant, plant_context, seed_points[i,:], iris_options)
    ellipse = hpoly.MaximumVolumeInscribedEllipsoid()
    print("Time: %6.2f \tVolume: %6.2f \tCenter:" % (time.time() - start_time, ellipse.Volume()),
          ellipse.center(), flush=True)
    regions.append(hpoly)

Time:   0.16 	Volume:   1.52 	Center: [ 0.19923802 -0.51555813 -0.16204011]
Time:   0.06 	Volume:   0.13 	Center: [ 0.67938601 -0.4962609   1.53735528]
Time:   0.02 	Volume:  24.21 	Center: [1.89511051e-09 1.88903337e-09 2.69925482e-09]
Time:   0.05 	Volume:   0.22 	Center: [-0.26468508 -0.33151235  1.54512993]


In [107]:
def inpolycheck(q0,q1,q2, A, b):
    q = np.array([q0, q1, q2])
    res = np.min(1.0*(A@q-b<=0))
    #print(res)
    return res

#create meshes for regions by sampling, sorry amice
meshes = []
col1 = 0x002BFF
col2 = 0x3EFF00 
idx = 0
step = 1/(1.0*len(regions))

for region in regions:
    A = region.A()
    b = region.b()
    col_hand = partial(inpolycheck, A=A, b=b)
    vertices, triangles = mcubes.marching_cubes_func(tuple(q_low), tuple(q_high), 90, 90, 90, col_hand, 0.5)
    regstr = "regions" +str(idx)
    col = + 1000*idx
    print(col)
    vis2[regstr].set_object(
            meshcat.geometry.TriangularMeshGeometry(vertices, triangles),
            meshcat.geometry.MeshLambertMaterial(color= int((1-idx*step)*col1 + idx*step*col2), wireframe=False))
    idx+=1

0
1000
2000
3000


In [82]:
q = seed_points[1,:]

np.min(1.0*(A@q-b<=0))


0.0

In [98]:
col1 = 0x002BFF
col2 = 0x3EFF00 
idx = 0

In [104]:
t = 0.1
col = 
col

422987